In [1]:
# Linking to drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install unbabel-comet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.2 MB/s 
     |████████████████████████████████| 9.5 MB 10.8 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 1.2 MB 53.4 MB/s 
     |████████████████████████████████| 4.4 MB 58.1 MB/s 
     |████████████████████████████████| 101 kB 14.5 MB/s 
     |████████████████████████████████| 409 kB 46.6 MB/s 
     |████████████████████████████████| 92 kB 13.5 MB/s 
     |████████████████████████████████| 585 kB 66.1 MB/s 
     |████████████████████████████████| 140 kB 75.0 MB/s 
     |████████████████████████████████| 596 kB 68.5 MB/s 
     |████████████████████████████████| 1.1 MB 59.7 MB/s 
     |████████████████████████████████| 6.6 MB 52.4 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████

In [ ]:
from comet import download_model, load_from_checkpoint

In [2]:
# Loading the drive
import os
os.chdir("/content/gdrive/Shareddrives/Low_Budget_MT")

In [3]:
# Importing needed libraries for preprocessing and visualization
from my_helpers import split_srctgt, generating_BPE, run_model

import warnings
warnings.filterwarnings('ignore')

In [4]:
#! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install . --use-feature=in-tree-build

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt
     |████████████████████████████████| 116 kB 24.7 MB/s 
     |████████████████████████████████| 507 kB 62.5 MB/s 
     |████████████████████████████████| 103 kB 62.1 MB/s 
     |████████████████████████████████| 264 kB 71.6 MB/s 
     |████████████████████████████████| 843 kB 58.7 MB/s 
     |████████████████████████████████| 57 kB 6.4 MB/s 
  Created wheel for joeynmt: filename=joeynmt-1.5.1-py3-none-any.whl size=86509 sha256=608b68876a691bf5bfdc690dbdb9c05acec60dee908ac5b31ecae40e9a1c2e25
  Stored in directory: /tmp/pip-ephem-wheel-cache-quvbpoqs/wheels/ec/b0/4e/e15e388dd23d01f1c2b4065e1bb36099e5e22abe4f7147f706
  Created wheel for wrapt: filename=wrapt-1.11.1-cp37-cp37m-linux_x86_64.whl size=68430 sha256=a2b08b41ad23b0b42b59eb566754ea3625e0d7565d367e3601d0f9295a4d8c85
  Stored in directory: /root/.cache/pip/wheels/4e/58/9d

In [5]:
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 9.1 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [ ]:
# Setting language and data file:
import pandas as pd
# 3. COMET
def comet(files, train_file, number):

  data = {}
  l  = []
  for i in files: 
    with open(i, "r") as src_file:
        x = src_file.read().splitlines()
        l.append(len(x))
        data[i] = x
  assert l.count(l[0]) == len(l), 'Check your files'

  df = pd.DataFrame(data)
  df.columns = ['source_sentence','target_sentence','scores']
  df["scores"] = pd.to_numeric(df["scores"])
  df = df.sort_values(by = ['scores'])
  print(df.head(1))

  return df.head(number), df.tail(df.shape[0]-number)

def comet_data_load(datafile, target_language, query, k, iteration):

  path = os.getcwd() 
  data_path = 'data/'+target_language
  train_file = '/content/gdrive/Shareddrives/Low_Budget_MT/data/sw/'+k+'/'+'swtrain.csv'
  fpath = os.path.join(path, data_path,k,query)
  print(path)
  print(data_path)
  print(fpath)

  if iteration== str(2):
    it = str(int(iteration)-1)
    train = pd.read_csv('/content/gdrive/Shareddrives/Low_Budget_MT/data/'+target_language+'/'+k+'/'+target_language+'train.csv')
    print(train.shape)
    t, rem = comet([fpath+'/1/rem.en', fpath+'/1/rem.'+target_language, fpath+'/1/scores'], train_file, 20000)
    print(t.shape, rem.shape)
    train_full = pd.concat([train,t])

    os.chdir(fpath)

    t.to_csv(iteration+'.csv', index=False)
    train_full.to_csv(query+'_train.csv', index=False)
    rem.to_csv(query+'_rem.csv',index=False)

    os.chdir(path)

  else:
    os.chdir(fpath)

    it = str(int(iteration)-1)

    train, rem = comet([fpath+'/'+it+'/rem.en', fpath+'/'+it+'/rem.'+target_language, fpath+'/'+it+'/scores'], train_file, 20000)
    print(train.shape)
    train.to_csv(iteration+'.csv', index=False)
    train.to_csv(query+'_train.csv',index=False, mode='a', header=False)
    rem.to_csv(query+'_rem.csv',index=False)

    os.chdir(path)

In [ ]:
def yhat(rem_file, src, tgt,k,iteration,M):
  path = '/content/gdrive/Shareddrives/Low_Budget_MT/data/'+tgt+'/'+k+'/comet/'+iteration
  df = pd.read_csv(rem_file)
  print(df.shape)

  os.chdir(path)
  print(os.getcwd())

  # Splitting x and y:
  os.environ['src'] = src
  os.environ['tgt']  = tgt
  with open("rem."+src, "w") as src_file, open("rem."+tgt, "w") as trg_file:
      for index, row in df.iterrows():
        src_file.write(row["source_sentence"]+"\n")
        trg_file.write(row["target_sentence"]+"\n")

  # Getting bpe for rem
  os.system(' subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < rem.$tgt > rem.bpe.$tgt')
  os.chdir('/content/gdrive/Shareddrives/Low_Budget_MT')
  print(os.getcwd())

  #translate rem
  os.environ["model_path"] = '/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/'+k+'/'+M+'/config.yaml'
  os.environ["x_path"] = path+'/rem.bpe.'+tgt
  os.environ["yhat_path"] = path+'/yhat.'+src

  os.system('cd joeynmt; python -m joeynmt translate $model_path < $x_path > $yhat_path')


def comet_scoring(rem_file, src, tgt,k,iteration,M):
  path = '/content/gdrive/Shareddrives/Low_Budget_MT/data/'+tgt+'/'+k+'/comet/'+iteration
  print('Starting here')
  with open(path+'/rem.'+src, "r") as x_file, open(path+'/rem.bpe.'+tgt, "r") as y_file, open(path+'/yhat.'+src, "r") as z_file:
    x = x_file.read().splitlines()
    y = y_file.read().splitlines()
    z = z_file.read().splitlines()
    print(len(x),len(y),len(z))

  with open(path+'/rem.'+tgt, "r") as a_file:
    a = a_file.read().splitlines()

    print(len(a))

  f = [i for i,j in enumerate(z) if len(j)==0]

  if len(f) !=0:
    x = [i for j, i in enumerate(x) if j not in f]
    y = [i for j, i in enumerate(y) if j not in f]
    z = [i for j, i in enumerate(z) if j not in f]
    a = [i for j, i in enumerate(a) if j not in f]
    
    print(len(x),len(y),len(z))
  
  with open(path+'/rem.'+src, "w") as x_file, open(path+'/rem.bpe.'+tgt, "w") as y_file, open(path+'/yhat.'+src, "w") as z_file:
    for i in zip(x,y,z):
        x_file.write(i[0]+"\n")
        y_file.write(i[1]+"\n")
        z_file.write(i[2]+"\n")

  with open(path+'/rem.'+tgt, "w") as a_file:
    for i in a:
        a_file.write(i+"\n")

  assert len(x)==len(a)==len(z), "Error in translating"

  data = [{'src':i[0], 'mt':i[1]} for i in zip(x,y)]

  model_path = download_model("wmt21-comet-qe-mqm")
  model = load_from_checkpoint(model_path)

  seg_scores, sys_score = model.predict(data, batch_size=8, gpus=1)

  print(len(seg_scores))

  with open(path+'/scores', "w") as x_file:
    for i in seg_scores:
        x_file.write(str(i)+"\n")

  if os.path.exists(path+'/scores'):
    print('done')
  else:
    print('error')

# Spanish COMET K1

## Iteration 5

### AL Strategy and preprocessing

In [ ]:
comet_data_load('estrain.csv', 'es', 'comet','k1', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/es
/content/gdrive/Shareddrives/Low_Budget_MT/data/es/k1/comet
(30000, 2)
                                         source_sentence  \
41750  I quote from The Guardian of 2 April, in which...   

                                         target_sentence    scores  
41750  Cito a Ingvar Carlsson, anterior Primer Minist... -0.006562  
(20000, 3) (80000, 3)


In [ ]:
split_srctgt('/data/es/k1/comet/comet_train.csv','en', 'es', 'comet','k1', '2')

(50000, 3)


In [ ]:
generating_BPE('en', 'es', 'comet', 'k1','2')

In [ ]:
# Swahili to English
run_model('es', 'es','en','transformer_reverse_enes2_comet','comet','k1','2')
! cd joeynmt; python3 -m joeynmt train configs/k1/transformer_reverse_enes2_comet.yaml

2022-09-01 17:59:53,750 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-01 17:59:54,119 - INFO - joeynmt.data - Loading training data...
2022-09-01 17:59:55,280 - INFO - joeynmt.data - Building vocabulary...
2022-09-01 17:59:55,577 - INFO - joeynmt.data - Loading dev data...
2022-09-01 17:59:55,598 - INFO - joeynmt.data - Loading test data...
2022-09-01 17:59:56,085 - INFO - joeynmt.data - Data loaded.
2022-09-01 17:59:56,086 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-01 17:59:56,321 - INFO - joeynmt.model - Enc-dec model built.
2022-09-01 18:00:00,842 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-01 18:00:01,306 - INFO - joeynmt.training - Total params: 12164352
2022-09-01 18:00:05,795 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enes2_comet
2022-09-01 18:00:05,796 - INFO - joeynmt.helpers -                       cfg.data.src : es
2022-09-01 18:00:05,796 - INFO - joeynmt.helpers -   

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/k1/transformer_reverse_enes2_comet/validations.txt"

Steps: 600	Loss: 188449.85938	PPL: 109.77616	bleu: 0.18784	LR: 0.00100000	*
Steps: 1200	Loss: 165517.95312	PPL: 61.97331	bleu: 0.85035	LR: 0.00100000	*
Steps: 1800	Loss: 149820.29688	PPL: 41.90179	bleu: 1.68466	LR: 0.00100000	*
Steps: 2400	Loss: 140163.46875	PPL: 32.93593	bleu: 2.61408	LR: 0.00100000	*
Steps: 3000	Loss: 132761.51562	PPL: 27.38562	bleu: 3.65862	LR: 0.00100000	*
Steps: 3600	Loss: 126930.74219	PPL: 23.68033	bleu: 4.06993	LR: 0.00100000	*
Steps: 4200	Loss: 121713.87500	PPL: 20.79219	bleu: 5.32460	LR: 0.00100000	*
Steps: 4800	Loss: 116936.70312	PPL: 18.45754	bleu: 6.65864	LR: 0.00100000	*
Steps: 5400	Loss: 113429.71875	PPL: 16.91222	bleu: 6.43419	LR: 0.00100000	*
Steps: 6000	Loss: 110292.19531	PPL: 15.63969	bleu: 8.22504	LR: 0.00100000	*
Steps: 6600	Loss: 107040.90625	PPL: 14.42194	bleu: 9.10509	LR: 0.00100000	*
Steps: 7200	Loss: 103888.64844	PPL: 13.33188	bleu: 10.01603	LR: 0.00100000	*
Steps: 7800	Loss: 101769.59375	PPL: 12.64581	bleu: 11.32277	LR: 0.00100000	*
Steps: 840

In [ ]:
# Reloading configuration file
ckpt_number = 58800
config = run_model('es', 'es','en','transformer_reverse_enes2_comet','comet','k1','2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_reverse_enes2_comet/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k1/transformer_reverse_enes2_comet"', f'model_dir: "models/k1/transformer_reverse_enes2_comet_reload"').replace(
        f'epochs: 60', f'epochs: 8')
        
with open("joeynmt/configs/k1/transformer_reverse_enes2_comet_reload.yaml",'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k1/transformer_reverse_enes2_comet_reload.yaml

2022-09-02 04:23:31,022 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-09-02 04:23:31,267 - INFO - joeynmt.data - Loading training data...
2022-09-02 04:23:34,387 - INFO - joeynmt.data - Building vocabulary...
2022-09-02 04:23:34,921 - INFO - joeynmt.data - Loading dev data...
2022-09-02 04:23:35,407 - INFO - joeynmt.data - Loading test data...
2022-09-02 04:23:36,470 - INFO - joeynmt.data - Data loaded.
2022-09-02 04:23:36,471 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-09-02 04:23:36,672 - INFO - joeynmt.model - Enc-dec model built.
2022-09-02 04:23:41,074 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-09-02 04:23:41,435 - INFO - joeynmt.training - Total params: 12164352
2022-09-02 04:23:44,968 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k1/transformer_reverse_enes2_comet/58800.ckpt
2022-09-02 04:23:48,347 - INFO - joeynmt.helpers -                           cfg.name : t

## Iteration 2

### AL Strategy and preprocessing

In [ ]:
comet_data_load('estrain.csv', 'es', 'comet','k1', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/es
/content/gdrive/Shareddrives/Low_Budget_MT/data/es/k1/comet
(30000, 2)
                                         source_sentence  \
41750  I quote from The Guardian of 2 April, in which...   

                                         target_sentence    scores  
41750  Cito a Ingvar Carlsson, anterior Primer Minist... -0.006562  
(5000, 3) (95000, 3)


In [ ]:
split_srctgt('/data/es/k1/comet/comet_train.csv','en', 'es', 'comet','k1', '2')

(35000, 3)


In [ ]:
generating_BPE('en', 'es', 'comet', 'k1','2')

In [ ]:
# Swahili to English
run_model('es', 'es','en','transformer_reverse_enes2_comet','comet','k1','2')
! cd joeynmt; python3 -m joeynmt train configs/k1/transformer_reverse_enes2_comet.yaml

2022-10-09 20:09:28,725 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-09 20:09:28,991 - INFO - joeynmt.data - Loading training data...
2022-10-09 20:09:29,666 - INFO - joeynmt.data - Building vocabulary...
2022-10-09 20:09:29,908 - INFO - joeynmt.data - Loading dev data...
2022-10-09 20:09:29,929 - INFO - joeynmt.data - Loading test data...
2022-10-09 20:09:30,349 - INFO - joeynmt.data - Data loaded.
2022-10-09 20:09:30,350 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-09 20:09:30,567 - INFO - joeynmt.model - Enc-dec model built.
2022-10-09 20:09:34,240 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-09 20:09:34,513 - INFO - joeynmt.training - Total params: 12160000
2022-10-09 20:09:38,172 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enes2_comet
2022-10-09 20:09:38,172 - INFO - joeynmt.helpers -                       cfg.data.src : es
2022-10-09 20:09:38,172 - INFO - joeynmt.helpers -   

In [ ]:
# Importing needed libraries 
from my_helpers import train_test, data_load, split_srctgt, generating_BPE, run_model, rttl_scoring, nrttl_data_load

import warnings
warnings.filterwarnings('ignore')

# Spanish COMET K2

## Iteration 5

### AL Strategy and preprocessing

In [ ]:
comet_data_load('estrain.csv', 'es', 'comet','k2', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/es
/content/gdrive/Shareddrives/Low_Budget_MT/data/es/k2/comet
(30000, 2)
                                         source_sentence  \
41593  I quote from The Guardian of 2 April, in which...   

                                         target_sentence   scores  
41593  Cito a Ingvar Carlsson, anterior Primer Minist... -0.00783  
(20000, 3) (80000, 3)


In [ ]:
split_srctgt('/data/es/k2/comet/comet_train.csv','en', 'es', 'comet','k2', '2')

(50000, 3)


In [ ]:
generating_BPE('en', 'es', 'comet', 'k2','2')

In [ ]:
# Swahili to English
run_model('es', 'es','en','transformer_reverse_enes2_comet','comet','k2','2')
! cd joeynmt; python3 -m joeynmt train configs/k2/transformer_reverse_enes2_comet.yaml

2022-10-15 01:04:51,796 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 01:04:52,069 - INFO - joeynmt.data - Loading training data...
2022-10-15 01:04:53,219 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 01:04:53,473 - INFO - joeynmt.data - Loading dev data...
2022-10-15 01:04:53,492 - INFO - joeynmt.data - Loading test data...
2022-10-15 01:04:53,979 - INFO - joeynmt.data - Data loaded.
2022-10-15 01:04:53,979 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 01:04:54,190 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 01:04:57,909 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 01:04:58,243 - INFO - joeynmt.training - Total params: 12163072
2022-10-15 01:05:01,992 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enes2_comet
2022-10-15 01:05:01,993 - INFO - joeynmt.helpers -                       cfg.data.src : es
2022-10-15 01:05:01,993 - INFO - joeynmt.helpers -   

In [6]:
# Output our validation accuracy
! cat "joeynmt/models/k2/transformer_reverse_enes2_comet/validations.txt"

Steps: 600	Loss: 183374.06250	PPL: 108.97009	bleu: 0.32569	LR: 0.00100000	*
Steps: 1200	Loss: 160091.21875	PPL: 60.06647	bleu: 0.79918	LR: 0.00100000	*
Steps: 1800	Loss: 146398.06250	PPL: 42.31552	bleu: 1.88925	LR: 0.00100000	*
Steps: 2400	Loss: 136250.92188	PPL: 32.64102	bleu: 2.48901	LR: 0.00100000	*
Steps: 3000	Loss: 129189.50781	PPL: 27.24649	bleu: 3.21288	LR: 0.00100000	*
Steps: 3600	Loss: 123532.29688	PPL: 23.57536	bleu: 4.06951	LR: 0.00100000	*
Steps: 4200	Loss: 118050.10156	PPL: 20.49040	bleu: 4.98408	LR: 0.00100000	*
Steps: 4800	Loss: 114333.44531	PPL: 18.63194	bleu: 5.77568	LR: 0.00100000	*
Steps: 5400	Loss: 109922.08594	PPL: 16.64360	bleu: 7.09986	LR: 0.00100000	*
Steps: 6000	Loss: 106638.42969	PPL: 15.30261	bleu: 8.17164	LR: 0.00100000	*
Steps: 6600	Loss: 103786.76562	PPL: 14.22601	bleu: 9.11134	LR: 0.00100000	*
Steps: 7200	Loss: 101311.17969	PPL: 13.35301	bleu: 9.71201	LR: 0.00100000	*
Steps: 7800	Loss: 98424.81250	PPL: 12.40256	bleu: 10.56274	LR: 0.00100000	*
Steps: 8400	

In [7]:
# Reloading configuration file
ckpt_number = 49200
config = run_model('es', 'es','en','transformer_reverse_enes2_comet','comet','k2','2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k2/transformer_reverse_enes2_comet/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k2/transformer_reverse_enes2_comet"', f'model_dir: "models/k2/transformer_reverse_enes2_comet_reload"').replace(
        f'epochs: 60', f'epochs: 17')
        
with open("joeynmt/configs/k2/transformer_reverse_enes2_comet_reload.yaml",'w') as f:
    f.write(reload_config)

In [8]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k2/transformer_reverse_enes2_comet_reload.yaml

2022-10-17 10:27:26,045 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-17 10:27:26,338 - INFO - joeynmt.data - Loading training data...
2022-10-17 10:27:29,309 - INFO - joeynmt.data - Building vocabulary...
2022-10-17 10:27:29,808 - INFO - joeynmt.data - Loading dev data...
2022-10-17 10:27:30,239 - INFO - joeynmt.data - Loading test data...
2022-10-17 10:27:31,295 - INFO - joeynmt.data - Data loaded.
2022-10-17 10:27:31,295 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-17 10:27:31,502 - INFO - joeynmt.model - Enc-dec model built.
2022-10-17 10:27:34,950 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-17 10:27:35,233 - INFO - joeynmt.training - Total params: 12163072
2022-10-17 10:27:38,823 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k2/transformer_reverse_enes2_comet/49200.ckpt
2022-10-17 10:27:42,069 - INFO - joeynmt.helpers -                           cfg.name : t

# Spanish COMET K3

In [ ]:
from my_helpers import comet_data_load

## Iteration 5

### AL Strategy and preprocessing

In [ ]:
comet_data_load('estrain.csv', 'es', 'comet','k3', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/es
/content/gdrive/Shareddrives/Low_Budget_MT/data/es/k3/comet
(30000, 2)
                                         source_sentence  \
41699  I quote from The Guardian of 2 April, in which...   

                                         target_sentence    scores  
41699  Cito a Ingvar Carlsson, anterior Primer Minist... -0.007404  
(20000, 3) (80000, 3)


In [ ]:
split_srctgt('/data/es/k3/comet/comet_train.csv','en', 'es', 'comet','k3', '2')

(50000, 3)


In [ ]:
generating_BPE('en', 'es', 'comet', 'k3','2')

In [ ]:
# Swahili to English
run_model('es', 'es','en','transformer_reverse_enes2_comet','comet','k3','2')
! cd joeynmt; python3 -m joeynmt train configs/k3/transformer_reverse_enes2_comet.yaml

2022-10-15 10:13:29,953 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 10:13:31,780 - INFO - joeynmt.data - Loading training data...
2022-10-15 10:13:32,833 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 10:13:33,078 - INFO - joeynmt.data - Loading dev data...
2022-10-15 10:13:33,097 - INFO - joeynmt.data - Loading test data...
2022-10-15 10:13:33,561 - INFO - joeynmt.data - Data loaded.
2022-10-15 10:13:33,561 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 10:13:33,770 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 10:13:37,440 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 10:13:37,710 - INFO - joeynmt.training - Total params: 12163072
2022-10-15 10:13:41,122 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enes2_comet
2022-10-15 10:13:41,122 - INFO - joeynmt.helpers -                       cfg.data.src : es
2022-10-15 10:13:41,122 - INFO - joeynmt.helpers -   

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/k3/transformer_reverse_enes2_comet/validations.txt"

Steps: 600	Loss: 184897.96875	PPL: 108.68339	bleu: 0.09846	LR: 0.00100000	*
Steps: 1200	Loss: 161000.17188	PPL: 59.29143	bleu: 0.89207	LR: 0.00100000	*
Steps: 1800	Loss: 147016.81250	PPL: 41.59127	bleu: 1.81816	LR: 0.00100000	*
Steps: 2400	Loss: 138131.32812	PPL: 33.20107	bleu: 2.99331	LR: 0.00100000	*
Steps: 3000	Loss: 130725.03125	PPL: 27.51637	bleu: 3.67601	LR: 0.00100000	*
Steps: 3600	Loss: 124889.36719	PPL: 23.73159	bleu: 4.48640	LR: 0.00100000	*
Steps: 4200	Loss: 120105.56250	PPL: 21.02064	bleu: 5.12784	LR: 0.00100000	*
Steps: 4800	Loss: 116165.03906	PPL: 19.02179	bleu: 6.28805	LR: 0.00100000	*
Steps: 5400	Loss: 112748.77344	PPL: 17.44336	bleu: 6.50329	LR: 0.00100000	*
Steps: 6000	Loss: 110024.95312	PPL: 16.27925	bleu: 7.84028	LR: 0.00100000	*
Steps: 6600	Loss: 107058.78125	PPL: 15.09976	bleu: 8.88119	LR: 0.00100000	*
Steps: 7200	Loss: 104575.39062	PPL: 14.17823	bleu: 8.95022	LR: 0.00100000	*
Steps: 7800	Loss: 102333.34375	PPL: 13.39466	bleu: 10.25958	LR: 0.00100000	*
Steps: 8400

In [ ]:
# Reloading configuration file
ckpt_number = 54000
config = run_model('es', 'es','en','transformer_reverse_enes2_comet','comet','k3','2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k3/transformer_reverse_enes2_comet/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k3/transformer_reverse_enes2_comet"', f'model_dir: "models/k3/transformer_reverse_enes2_comet_reload"').replace(
        f'epochs: 60', f'epochs: 13')
        
with open("joeynmt/configs/k3/transformer_reverse_enes2_comet_reload.yaml",'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k3/transformer_reverse_enes2_comet_reload.yaml

2022-10-16 20:00:04,329 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-16 20:00:05,265 - INFO - joeynmt.data - Loading training data...
2022-10-16 20:00:11,452 - INFO - joeynmt.data - Building vocabulary...
2022-10-16 20:00:12,347 - INFO - joeynmt.data - Loading dev data...
2022-10-16 20:00:14,000 - INFO - joeynmt.data - Loading test data...
2022-10-16 20:00:16,155 - INFO - joeynmt.data - Data loaded.
2022-10-16 20:00:16,156 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-16 20:00:16,364 - INFO - joeynmt.model - Enc-dec model built.
2022-10-16 20:00:19,512 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-16 20:00:19,776 - INFO - joeynmt.training - Total params: 12163072
2022-10-16 20:00:23,308 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k3/transformer_reverse_enes2_comet/54000.ckpt
2022-10-16 20:00:28,170 - INFO - joeynmt.helpers -                           cfg.name : t

# Spanish COMET K4

In [ ]:
from my_helpers import comet_data_load

## Iteration 5

### AL Strategy and preprocessing

In [ ]:
comet_data_load('estrain.csv', 'es', 'comet','k4', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/es
/content/gdrive/Shareddrives/Low_Budget_MT/data/es/k4/comet
(30000, 2)
                                         source_sentence  \
41776  I quote from The Guardian of 2 April, in which...   

                                         target_sentence    scores  
41776  Cito a Ingvar Carlsson, anterior Primer Minist... -0.000063  
(20000, 3) (80000, 3)


In [ ]:
split_srctgt('/data/es/k4/comet/comet_train.csv','en', 'es', 'comet','k4', '2')

(50000, 3)


In [ ]:
generating_BPE('en', 'es', 'comet', 'k4','2')

In [ ]:
# Swahili to English
run_model('es', 'es','en','transformer_reverse_enes2_comet','comet','k4','2')
! cd joeynmt; python3 -m joeynmt train configs/k4/transformer_reverse_enes2_comet.yaml

2022-10-15 10:15:08,139 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 10:15:08,397 - INFO - joeynmt.data - Loading training data...
2022-10-15 10:15:09,488 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 10:15:09,754 - INFO - joeynmt.data - Loading dev data...
2022-10-15 10:15:09,773 - INFO - joeynmt.data - Loading test data...
2022-10-15 10:15:10,252 - INFO - joeynmt.data - Data loaded.
2022-10-15 10:15:10,252 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 10:15:10,459 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 10:15:13,777 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 10:15:14,039 - INFO - joeynmt.training - Total params: 12165376
2022-10-15 10:15:17,738 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enes2_comet
2022-10-15 10:15:17,738 - INFO - joeynmt.helpers -                       cfg.data.src : es
2022-10-15 10:15:17,739 - INFO - joeynmt.helpers -   

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/k4/transformer_reverse_enes2_comet/validations.txt"

Steps: 600	Loss: 182173.60938	PPL: 106.05341	bleu: 0.21452	LR: 0.00100000	*
Steps: 1200	Loss: 159597.50000	PPL: 59.49888	bleu: 1.06764	LR: 0.00100000	*
Steps: 1800	Loss: 144216.67188	PPL: 40.13237	bleu: 2.30963	LR: 0.00100000	*
Steps: 2400	Loss: 134268.95312	PPL: 31.10925	bleu: 2.63150	LR: 0.00100000	*
Steps: 3000	Loss: 126429.45312	PPL: 25.45218	bleu: 4.01953	LR: 0.00100000	*
Steps: 3600	Loss: 120595.96094	PPL: 21.92120	bleu: 4.65902	LR: 0.00100000	*
Steps: 4200	Loss: 115665.06250	PPL: 19.32144	bleu: 5.76962	LR: 0.00100000	*
Steps: 4800	Loss: 111259.95312	PPL: 17.26078	bleu: 6.84756	LR: 0.00100000	*
Steps: 5400	Loss: 108313.67969	PPL: 16.00670	bleu: 7.71303	LR: 0.00100000	*
Steps: 6000	Loss: 104840.49219	PPL: 14.64485	bleu: 9.61404	LR: 0.00100000	*
Steps: 6600	Loss: 101542.92188	PPL: 13.45923	bleu: 9.96378	LR: 0.00100000	*
Steps: 7200	Loss: 99469.65625	PPL: 12.76346	bleu: 10.81158	LR: 0.00100000	*
Steps: 7800	Loss: 96695.59375	PPL: 11.88843	bleu: 11.57697	LR: 0.00100000	*
Steps: 8400	

In [ ]:
# Reloading configuration file
ckpt_number = 55800
config = run_model('es', 'es','en','transformer_reverse_enes2_comet','comet','k4','2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k4/transformer_reverse_enes2_comet/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k4/transformer_reverse_enes2_comet"', f'model_dir: "models/k4/transformer_reverse_enes2_comet_reload"').replace(
        f'epochs: 60', f'epochs: 11')
        
with open("joeynmt/configs/k4/transformer_reverse_enes2_comet_reload.yaml",'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k4/transformer_reverse_enes2_comet_reload.yaml

2022-10-16 19:53:33,366 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-16 19:53:33,776 - INFO - joeynmt.data - Loading training data...
2022-10-16 19:53:37,846 - INFO - joeynmt.data - Building vocabulary...
2022-10-16 19:53:38,525 - INFO - joeynmt.data - Loading dev data...
2022-10-16 19:53:39,306 - INFO - joeynmt.data - Loading test data...
2022-10-16 19:53:40,567 - INFO - joeynmt.data - Data loaded.
2022-10-16 19:53:40,567 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-16 19:53:40,784 - INFO - joeynmt.model - Enc-dec model built.
2022-10-16 19:53:43,896 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-16 19:53:44,161 - INFO - joeynmt.training - Total params: 12165376
2022-10-16 19:53:47,782 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k4/transformer_reverse_enes2_comet/55800.ckpt
2022-10-16 19:53:50,905 - INFO - joeynmt.helpers -                           cfg.name : t

# Spanish COMET K5

In [ ]:
from my_helpers import comet_data_load

In [ ]:
# Importing needed libraries for preprocessing and visualization
from my_helpers import rttl_data_load, split_srctgt, generating_BPE, run_model, rttl_scoring, nrttl_data_load

import warnings
warnings.filterwarnings('ignore')

## Iteration 5

### AL Strategy and preprocessing

In [ ]:
comet_data_load('estrain.csv', 'es', 'comet','k5', '2')

/content/gdrive/Shareddrives/Low_Budget_MT
data/es
/content/gdrive/Shareddrives/Low_Budget_MT/data/es/k5/comet
(30000, 2)
                                         source_sentence  \
89292  I quote from The Guardian of 2 April, in which...   

                                         target_sentence    scores  
89292  Cito a Ingvar Carlsson, anterior Primer Minist... -0.006542  
(20000, 3) (80000, 3)


In [ ]:
split_srctgt('/data/es/k5/comet/comet_train.csv','en', 'es', 'comet','k5', '2')

(50000, 3)


In [ ]:
generating_BPE('en', 'es', 'comet', 'k5','2')

In [ ]:
# Swahili to English
run_model('es', 'es','en','transformer_reverse_enes2_comet','comet','k5','2')
! cd joeynmt; python3 -m joeynmt train configs/k5/transformer_reverse_enes2_comet.yaml

2022-10-15 10:20:09,041 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-15 10:20:09,362 - INFO - joeynmt.data - Loading training data...
2022-10-15 10:20:10,491 - INFO - joeynmt.data - Building vocabulary...
2022-10-15 10:20:10,797 - INFO - joeynmt.data - Loading dev data...
2022-10-15 10:20:10,817 - INFO - joeynmt.data - Loading test data...
2022-10-15 10:20:11,307 - INFO - joeynmt.data - Data loaded.
2022-10-15 10:20:11,308 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-15 10:20:11,522 - INFO - joeynmt.model - Enc-dec model built.
2022-10-15 10:20:14,956 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-15 10:20:15,238 - INFO - joeynmt.training - Total params: 12164352
2022-10-15 10:20:19,080 - INFO - joeynmt.helpers -                           cfg.name : transformer_reverse_enes2_comet
2022-10-15 10:20:19,080 - INFO - joeynmt.helpers -                       cfg.data.src : es
2022-10-15 10:20:19,081 - INFO - joeynmt.helpers -   

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/k5/transformer_reverse_enes2_comet/validations.txt"

Steps: 600	Loss: 184052.35938	PPL: 104.53773	bleu: 0.15703	LR: 0.00100000	*
Steps: 1200	Loss: 162707.20312	PPL: 60.96652	bleu: 1.05774	LR: 0.00100000	*
Steps: 1800	Loss: 147564.56250	PPL: 41.58709	bleu: 2.11358	LR: 0.00100000	*
Steps: 2400	Loss: 138241.37500	PPL: 32.86035	bleu: 2.96683	LR: 0.00100000	*
Steps: 3000	Loss: 130247.46094	PPL: 26.85155	bleu: 4.14738	LR: 0.00100000	*
Steps: 3600	Loss: 124779.66406	PPL: 23.38736	bleu: 4.57077	LR: 0.00100000	*
Steps: 4200	Loss: 119580.66406	PPL: 20.50888	bleu: 4.88103	LR: 0.00100000	*
Steps: 4800	Loss: 115548.53125	PPL: 18.52271	bleu: 5.48943	LR: 0.00100000	*
Steps: 5400	Loss: 111971.32812	PPL: 16.92227	bleu: 6.98340	LR: 0.00100000	*
Steps: 6000	Loss: 108647.76562	PPL: 15.55948	bleu: 8.26344	LR: 0.00100000	*
Steps: 6600	Loss: 105749.51562	PPL: 14.46098	bleu: 8.55365	LR: 0.00100000	*
Steps: 7200	Loss: 103078.02344	PPL: 13.51725	bleu: 9.35082	LR: 0.00100000	*
Steps: 7800	Loss: 100545.08594	PPL: 12.67941	bleu: 10.21964	LR: 0.00100000	*
Steps: 8400

In [ ]:
# Reloading configuration file
ckpt_number = 53400
config = run_model('es', 'es','en','transformer_reverse_enes2_comet','comet','k5','2')
reload_config = config.replace(
    f'#load_model: path ', 
    f'load_model: "/content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k5/transformer_reverse_enes2_comet/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/k5/transformer_reverse_enes2_comet"', f'model_dir: "models/k5/transformer_reverse_enes2_comet_reload"').replace(
        f'epochs: 60', f'epochs: 13')
        
with open("joeynmt/configs/k5/transformer_reverse_enes2_comet_reload.yaml",'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/k5/transformer_reverse_enes2_comet_reload.yaml

2022-10-16 19:56:10,112 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-10-16 19:56:10,553 - INFO - joeynmt.data - Loading training data...
2022-10-16 19:56:15,375 - INFO - joeynmt.data - Building vocabulary...
2022-10-16 19:56:16,563 - INFO - joeynmt.data - Loading dev data...
2022-10-16 19:56:17,540 - INFO - joeynmt.data - Loading test data...
2022-10-16 19:56:19,344 - INFO - joeynmt.data - Data loaded.
2022-10-16 19:56:19,344 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-10-16 19:56:19,558 - INFO - joeynmt.model - Enc-dec model built.
2022-10-16 19:56:22,795 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
2022-10-16 19:56:23,069 - INFO - joeynmt.training - Total params: 12164352
2022-10-16 19:56:26,853 - INFO - joeynmt.training - Loading model from /content/gdrive/Shareddrives/Low_Budget_MT/joeynmt/models/k5/transformer_reverse_enes2_comet/53400.ckpt
2022-10-16 19:56:31,075 - INFO - joeynmt.helpers -                           cfg.name : t